In [1]:
from datasets import load_dataset
import config

import os
os.environ['CUDA_VISIBLE_DEVICE']='0,1,2,3,4,5,6,7'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
dataset = load_dataset(
    "./data/ok_vqa", split="train", data_dir=config.ok_vqa_dir, images_dir=config.coco_dir,
    trust_remote_code=True
)

/data1/jyc/miniconda3/envs/icl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models import Idefics
import torch
device = torch.device("cuda:6")
model = Idefics(config.idefics_9b_path, precision="bf16", device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 19/19 [00:17<00:00,  1.07it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 6; 23.69 GiB total capacity; 3.75 GiB already allocated; 82.94 MiB free; 3.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from data import prepare_dataloader


dataloader = prepare_dataloader(
    dataset,
    1,
    2,
    instruction="Provide an answer to the question. Use the image to answer.",
)

In [ ]:
import PIL
template = (
    "{% if messages[0]['role'] == 'instruction' %}"
                "Instruction: {{ messages[0]['content'] }}\n"
                "{% set messages = messages[1:] %}"
            "{% endif %}"
            "{% for message in messages %}"
                "Question:" 
                "{% for line in message['query'] %}"
                    "{% if line['type'] == 'text' %}"
                        "{{- line['text'] }}"
                    "{% elif line['type'] == 'image' %}"
                        "{{ '<image>' }}"
                    "{% endif %}"
                "{% endfor %}"
                "<end_of_utterance>\n"
                "{% if 'answer' in message %}"
                    "Short answer: " 
                    "{% for line in message['answer'] %}"
                        "{% if line['type'] == 'text' %}"
                            "{{- line['text'] }}"
                        "{% elif line['type'] == 'image' %}"
                            "{{ '<image>' }}"
                        "{% endif %}"
                    "{% endfor %}"
                    "<end_of_utterance>\n"
                "{% endif %}"
            "{% endfor %}"
            "{% if add_generation_prompt %}"
                "Short answer: " 
            "{% endif %}"
)

for batch in dataloader:
    images, text, answer = batch
    # print(model.generate(text, images, ))
    
    texts = model.apply_prompt_template(text, template, add_generation_prompt=True)
    print(text)
    inputs = []
    for i, (text, image_list) in enumerate(zip(texts, images)):
        text = text.split("<image>")
        result = []
        if len(text) - 1 != len(image_list):
            raise ValueError(
                f"In the {i}-th input, the number of images does not match the number of image tokens in the text."
            )
        for seg, image in zip(text, image_list):
            if seg != "":
                result.append(seg)
            result.append(image)
        if text[-1] != "":
            result.append(text[-1])
        inputs.append(result + text[-1])
        print(result)
    print(model._generate({"prompts": inputs}, {"max_new_tokens": 20, "num_beams": 3}))
    print(answer)
    break

[[{'role': 'instruction', 'content': 'Provide an answer to the question. Use the image to answer.'}, {'role': 'example', 'query': [{'type': 'image'}, {'type': 'text', 'text': 'What is the hairstyle of the blond called?'}], 'answer': [{'type': 'text', 'text': 'pony tail'}]}, {'role': 'example', 'query': [{'type': 'image'}, {'type': 'text', 'text': 'How old do you have to be in canada to do this?'}], 'answer': [{'type': 'text', 'text': '18'}]}, {'role': 'question', 'query': [{'type': 'image'}, {'type': 'text', 'text': 'Can you guess the place where the man is playing?'}]}]]
['Instruction: Provide an answer to the question. Use the image to answer.\nQuestion:', <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x479 at 0x7F877C9FAA70>, 'What is the hairstyle of the blond called?<end_of_utterance>\nShort answer: pony tail<end_of_utterance>\nQuestion:', <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x428 at 0x7F877C9F8FD0>, 'How old do you have to be in canada to do this?<

In [ ]:
print(text[0])

Instruction: Provide an answer to the question. Use the image to answer.Question:
